## Import Required Libraries

In [1]:
import duckdb

In [35]:
duckdb.sql("""           
           WITH home_players2 AS(
               SELECT UNNEST(home_players) AS home_players
               FROM read_json('Game Statistics/0022400066_2024-10-23.jsonl', auto_detect=True)
           ),
           
           home_player_stats AS(
               SELECT home_players.statistics AS player_stats
               FROM home_players2
           )
           SELECT player_stats.*
           FROM home_player_stats""")

┌─────────┬────────┬────────────────┬───┬───────────┬──────────────────────┬─────────────────┬──────────────────────┐
│ assists │ blocks │ blocksReceived │ … │ turnovers │ twoPointersAttempted │ twoPointersMade │ twoPointersPercent…  │
│ uint64  │ uint64 │     uint64     │   │  uint64   │        uint64        │     uint64      │        double        │
├─────────┼────────┼────────────────┼───┼───────────┼──────────────────────┼─────────────────┼──────────────────────┤
│       1 │      0 │              3 │ … │         1 │                   12 │               7 │    0.583333333333333 │
│       0 │      0 │              0 │ … │         0 │                    3 │               1 │    0.333333333333333 │
│       0 │      0 │              0 │ … │         1 │                    7 │               4 │   0.5714285714285711 │
│       1 │      0 │              3 │ … │         0 │                    4 │               1 │                 0.25 │
│       3 │      1 │              2 │ … │         1 │   

In [88]:
home_stats = duckdb.sql("""
           WITH home_players AS(
               SELECT UNNEST(home_players) AS home_players
               FROM read_json('Game Statistics/0022400066_2024-10-23.jsonl', auto_detect=True)
           ),
           
           home_player_stats AS(
               SELECT home_players.statistics AS player_stats,
               home_players.name AS player_name
               FROM home_players
           )
           
           SELECT 
                home_team.team_name AS team_name,
                ROUND((home_team.points/(home_team.fieldGoalsAttempted + (0.44 * home_team.freeThrowsAttempted) - home_team.reboundsOffensive + home_team.turnovers))*100, 2) AS 'Offensive Rating',
                ROUND((away_team.points/(away_team.fieldGoalsAttempted + (0.44 * away_team.freeThrowsAttempted) - away_team.reboundsOffensive + away_team.turnovers))*100, 2) AS 'Defensive Rating',
                ROUND((home_team.points/(home_team.fieldGoalsAttempted + (0.44 * home_team.freeThrowsAttempted) - home_team.reboundsOffensive + home_team.turnovers))*100, 2) - ROUND((away_team.points/(away_team.fieldGoalsAttempted + (0.44 * away_team.freeThrowsAttempted) - away_team.reboundsOffensive + away_team.turnovers))*100, 2) AS 'Net Rating',
                home_team.pointsInThePaint AS 'Points In The Paint',
                home_team.pointsSecondChance AS 'Second Chance Points',
                home_team.pointsFromTurnovers AS 'Points From Turnovers',
                ROUND(home_team.fieldGoalsEffectiveAdjusted, 4)*100 AS 'Effective FG %',
                ROUND(home_team.trueShootingPercentage, 4)*100 AS 'True Shooting %',
                home_team.assistsTurnoverRatio AS 'Assist-Turnover Ratio',
                player_name AS 'Team MVP', 
                MAX((player_stats.points::FLOAT + player_stats.fieldGoalsMade::FLOAT + player_stats.freeThrowsMade::FLOAT - player_stats.fieldGoalsAttempted::FLOAT - player_stats.freeThrowsAttempted::FLOAT + player_stats.reboundsDefensive::FLOAT + (player_stats.reboundsOffensive::FLOAT/2::FLOAT) + player_stats.assists::FLOAT + player_stats.steals::FLOAT + (player_stats.blocks::FLOAT/2::FLOAT) - player_stats.foulsPersonal::FLOAT - player_stats.turnovers::FLOAT)::FLOAT / (home_team.points + home_team.fieldGoalsMade + home_team.freeThrowsMade - home_team.fieldGoalsAttempted - home_team.freeThrowsAttempted + home_team.reboundsDefensive + (home_team.reboundsOffensive::FLOAT/2::FLOAT) + home_team.assists + home_team.steals + (home_team.blocks::FLOAT/2::FLOAT) - home_team.foulsPersonal - home_team.turnovers)::FLOAT) AS 'PIE'
            FROM read_json('Game Statistics/0022400066_2024-10-23.jsonl', auto_detect=True)
            CROSS JOIN home_player_stats
            GROUP BY home_team, away_team, player_name
            ORDER BY PIE DESC
            LIMIT 1""").to_df()
home_stats

,team_name,Offensive Rating,Defensive Rating,Net Rating,Points In The Paint,Second Chance Points,Points From Turnovers,Effective FG %,True Shooting %,Assist-Turnover Ratio,Team MVP,PIE
0,76ers,105.95,124.45,-18.5,56,17,18,45.92,49.6,1.8,Kyle Lowry,0.244604


In [90]:
duckdb.sql("SELECT 'Action'")

┌──────────┐
│ 'Action' │
│ varchar  │
├──────────┤
│ Action   │
└──────────┘